# Stop Trader

The goal of the `stop_trader` is to explore whether it makes sense to set up stop-losses (or stop-buys) when entering a long position (or when selling everything).

Imagine that you buy a certain amount $a$ of a cryptocurrency (e.g. `BTC`) at price $p$. Is it a good idea to set up a stop-loss at some lower price $p' < p$? If so, how to determine the price $p'$? What if later the price $p$ goes up? Should we then adjust (increase) the stop-loss price $p'$ as well? These are some of the questions that we would like to answer by evaluating many instances of the `stop_trader` over the `BTCUSD` price history.

At any given moment the trader has (on its account) some amount $b$ of a base (crypto)currency (e.g. `BTC`) and some amount $q$ of a quote currency (e.g. `USD`). Let $p$ denote the base (crypto)currency price (e.g. `BTCUSD` price). If $b * p \ge q$ we say that the trader is in the `LONG` state. Otherwise, we say that the trader is in the `CASH` state.

* If the trader gets into the `LONG` state we define the stop-loss at the price $p' := (1 - \alpha) * p < p$, where $\alpha \in (0; 1)$ is a trader hyper-parameter (called `stop_order_margin`). As long as the trader is in the `LONG` state we only allow the price of the stop-loss $p'$ to increase over time. When the price $p$ drops and hits the stop-loss the trader automatically switches to the `CASH` state (by selling all its $b$ base (crypto)currency at price $p'$).
* Similarly, if the trader gets into the `CASH` state we define the stop-buy at the price $p' := (1 + \alpha) * p > p$. As long as the trader is in the `CASH` state we only allow the price of the stop-buy $p'$ to decrease over time. When the price $p$ increases and hits the stop-buy the trader automatically switches to the `LONG` state (by buying as much as possible base (crypto)currency at price $p'$).

One can easily observe that if there were no transaction fees and if we could always execute the stop-sells (stop-buys) at exactly the specified price $p'$, this strategy would never be worse than the Buy-And-HODL strategy. Moreover, we would (under these ideal circumstances) never loose any money (only potentially gain). Note: If you think that these assumptions (no transaction fees etc.) are far-fetched then take a look at the assumptions behind the famous [Black-Scholes model](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model) (they are even more unrealistic).

Unfortunately, we do have non-negligible transaction fees and the stop orders are (usually) not executed at exactly the desired price $p'$, but at a worse price (especially if there is not enough liquidity). Therefore, we need to tune the hyper-parameters of the trader so that we do not incur too many transaction fees and still stay profitable.

In addition to the above-mentioned `stop_order_margin` $\alpha$ we have the following hyper-parameters:

* `stop_order_move_margin` $\beta$: (A) If the trader is in the `LONG` state and the stop-loss price: $p' \le (1 - \beta) * p$ we allow the stop-loss price to increase. (B) Similarly, if the trader is in the `CASH` state and the stop-buy price $p' \ge (1 + \beta) * p$ we allow the stop-buy price to decrease.
* `stop_order_increase_per_day` $\gamma_+$: In the case (A) we allow the stop-loss price to increase by at most $\gamma_+ * 100\%$ per day, although we do not allow the stop-loss price to exceed the $(1 - \beta) * p$ threshold.
* `stop_order_decrease_per_day` $\gamma_-$: In the case (B) we allow the stop-buy price to decrease by at most $\gamma_- * 100\%$ per day, although we do not allow the stop-buy price to drop below the $(1 + \beta) * p$ threshold.

Batch evaluation of the `stop_trader` reveals that setting $\alpha = \beta = 0.1$, $\gamma_+ = 0.01$, and $\gamma_- = 0.1$ gives a reasonable performance (although there are many other settings which perform similarly). With these specific parameters the trader performs as follows (evaluated over 6 month intervals):

```
[2017-01-01 00:00:00 - 2017-07-01 00:00:00): 0.745228
[2017-02-01 00:00:00 - 2017-08-01 00:00:00): 0.876014
[2017-03-01 00:00:00 - 2017-09-01 00:00:00): 0.893903
[2017-04-01 00:00:00 - 2017-10-01 00:00:00): 1.02772
[2017-05-01 00:00:00 - 2017-11-01 00:00:00): 1.03472
[2017-06-01 00:00:00 - 2017-12-01 00:00:00): 0.90212
[2017-07-01 00:00:00 - 2018-01-01 00:00:00): 1.0353
[2017-08-01 00:00:00 - 2018-02-01 00:00:00): 0.771577
[2017-09-01 00:00:00 - 2018-03-01 00:00:00): 0.92533
[2017-10-01 00:00:00 - 2018-04-01 00:00:00): 0.877512
[2017-11-01 00:00:00 - 2018-05-01 00:00:00): 0.918183
[2017-12-01 00:00:00 - 2018-06-01 00:00:00): 1.00585
[2018-01-01 00:00:00 - 2018-07-01 00:00:00): 1.1751
[2018-02-01 00:00:00 - 2018-08-01 00:00:00): 1.25298
[2018-03-01 00:00:00 - 2018-09-01 00:00:00): 1.33426
[2018-04-01 00:00:00 - 2018-10-01 00:00:00): 1.28329
[2018-05-01 00:00:00 - 2018-11-01 00:00:00): 1.17252
[2018-06-01 00:00:00 - 2018-12-01 00:00:00): 1.40445
[2018-07-01 00:00:00 - 2019-01-01 00:00:00): 1.4096
[2018-08-01 00:00:00 - 2019-02-01 00:00:00): 1.45463
[2018-09-01 00:00:00 - 2019-03-01 00:00:00): 1.27935
[2018-10-01 00:00:00 - 2019-04-01 00:00:00): 1.46851
[2018-11-01 00:00:00 - 2019-05-01 00:00:00): 1.44526
[2018-12-01 00:00:00 - 2019-06-01 00:00:00): 0.987273
[2019-01-01 00:00:00 - 2019-07-01 00:00:00): 0.94727
[2019-02-01 00:00:00 - 2019-08-01 00:00:00): 0.876466
[2019-03-01 00:00:00 - 2019-09-01 00:00:00): 0.807884
[2019-04-01 00:00:00 - 2019-10-01 00:00:00): 0.856121
[2019-05-01 00:00:00 - 2019-11-01 00:00:00): 0.754844
[2019-06-01 00:00:00 - 2019-12-01 00:00:00): 0.685491
[2019-07-01 00:00:00 - 2020-01-01 00:00:00): 0.726241
[2019-08-01 00:00:00 - 2020-02-01 00:00:00): 0.91875
[2019-09-01 00:00:00 - 2020-03-01 00:00:00): 1.10986
[2019-10-01 00:00:00 - 2020-04-01 00:00:00): 1.42124
[2019-11-01 00:00:00 - 2020-05-01 00:00:00): 1.56454
[2019-12-01 00:00:00 - 2020-06-01 00:00:00): 1.39337
[2020-01-01 00:00:00 - 2020-07-01 00:00:00): 1.43864
[2020-02-01 00:00:00 - 2020-08-01 00:00:00): 1.33628
[2020-03-01 00:00:00 - 2020-09-01 00:00:00): 1.17149
[2020-04-01 00:00:00 - 2020-10-01 00:00:00): 0.860107
[2020-05-01 00:00:00 - 2020-11-01 00:00:00): 0.681354
[2020-06-01 00:00:00 - 2020-12-01 00:00:00): 0.860026
```

As you can see, the `stop_trader` sometimes beats the Buy-And-HODL strategy (although not always). Over the whole period:

```
[2017-01-01 00:00:00 - 2020-12-01 00:00:00): 1.10669
```

the `stop_trader` would beat the Buy-And-HODL strategy only by 10%. That being said, there are many other (suboptimal) choices of the hyper-parameters that would lead to much worse performance. It is questionable whether an average person would be able to set the stop-losses / stop-buys better than this algorithm. If not then an average person would be better off with HODLing without any stop-losses.